In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../joyful-pandas-master/data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


# 一、透视表（长变宽）

将类别特征的水平作为新的列

## 1、pivot

In [3]:
df.pivot(index='ID',columns='Gender',values='Math').head()

Gender,F,M
ID,,
1101,NaN,34.0
1102,32.5,NaN
1103,NaN,87.2
1104,80.4,NaN
1105,84.8,NaN


### pivot功能较少，并且索引列不能有重复值，但速度更快

以下语句会报错
```python
df.pivot(index='Class',columns='Gender',values='Math').head()
```

## 2、pivot_table

索引不必为非重复值，因此可以设为类别变量，做分组统计，默认的聚合函数为mean

### aggfunc：对组内进行聚合统计，默认mean

In [7]:
df.pivot_table(index='School',columns='Gender',values='Height',aggfunc=['mean','sum'])

mean               sum      
Gender           F           M     F     M
School                                    
S_1     173.125000  178.714286  1385  1251
S_2     173.727273  172.000000  1911  1548

### margins：汇总边际状态

In [9]:
df.pivot_table(index='School',columns='Gender',values='Height',aggfunc=['mean','sum'],margins=True)

mean                           sum            
Gender           F           M         All     F     M   All
School                                                      
S_1     173.125000  178.714286  175.733333  1385  1251  2636
S_2     173.727273  172.000000  172.950000  1911  1548  3459
All     173.473684  174.937500  174.142857  3296  2799  6095

### 行、列、值都可以为多级

In [10]:
df.pivot_table(index=['School','Class'],
              columns = ['Gender','Address'],
               values = ['Math','Height']
              )

Height                                                        \
Gender              F                                                     M   
Address      street_1 street_2 street_4 street_5 street_6 street_7 street_1   
School Class                                                                  
S_1    C_1        NaN    179.5    159.0      NaN      NaN      NaN    173.0   
       C_2        NaN      NaN    176.0    162.0    167.0      NaN      NaN   
       C_3      175.0      NaN      NaN    187.0      NaN      NaN      NaN   
S_2    C_1        NaN      NaN      NaN    159.0    161.0      NaN      NaN   
       C_2        NaN      NaN      NaN      NaN      NaN    188.5    175.0   
       C_3        NaN      NaN    157.0      NaN    164.0    190.0      NaN   
       C_4        NaN    176.0      NaN      NaN    175.5      NaN      NaN   

                                          ...        Math                    \
Gender                                    ...           F                     
Address      street_2 street_4 street_5   ...    street_4 street_5 street_6   
School Class                              ...                                 
S_1    C_1      186.0      NaN      NaN   ...        84.8      NaN      NaN   
       C_2        NaN      NaN    188.0   ...        63.5     33.8    68.40   
       C_3      195.0    161.0      NaN   ...         NaN     61.7      NaN   
S_2    C_1        NaN    163.5      NaN   ...         NaN     72.2    50.60   
       C_2        NaN    155.0    193.0   ...         NaN      NaN      NaN   
       C_3        NaN    187.0    171.0   ...        72.3      NaN    95.50   
       C_4        NaN      NaN      NaN   ...         NaN      NaN    53.65   

                                                                             
Gender                       M                                               
Address      street_7 street_1 street_2 street_4 street_5 street_6 street_7  
School Class                                                                 
S_1    C_1        NaN     34.0     87.2      NaN      NaN      NaN      NaN  
       C_2        NaN      NaN      NaN      NaN     97.0     58.8      NaN  
       C_3        NaN      NaN     85.2    31.50      NaN      NaN     49.7  
S_2    C_1        NaN      NaN      NaN    43.35      NaN      NaN     83.3  
       C_2      76.95     47.2      NaN    73.80     39.1      NaN      NaN  
       C_3      65.90      NaN      NaN    48.90     32.7      NaN      NaN  
       C_4        NaN      NaN      NaN      NaN      NaN      NaN     48.7  

[7 rows x 24 columns]

## 3、crosstab（交叉表）

- 是一种特殊的pivot_table，做分组统计，即统计频数，此时values默认为1

- 不支持多级分组


### values和aggfunc：必须同时出现，分组对数据进行聚合操作

默认values=1，aggfunc='count'

In [16]:
pd.crosstab(index = df['School'],
            columns=df.Gender,
            values=df.Math,
            aggfunc='mean')

Gender,F,M
School,,
S_1,64.100000,63.342857
S_2,66.427273,51.155556


### normalize参数：可选'all','index','columns'参数值

In [22]:
pd.crosstab(index = df['School'],
            columns=df.Gender,
            values=df.Math,
            aggfunc='mean',
            margins = True,normalize = 'index')

Gender,F,M
School,,
S_1,0.502971,0.497029
S_2,0.564940,0.435060
All,0.536740,0.463260


# 二、melt（宽变长）

### id_vars:需要保留的列；value_vars：需要stack的一组列

In [23]:
df1 = df.pivot(index = 'ID',columns='School',values = 'Math')
display(df1.head())

School,S_1,S_2
ID,,
1101,34.0,NaN
1102,32.5,NaN
1103,87.2,NaN
1104,80.4,NaN
1105,84.8,NaN


**melt之后会有NA值：**

In [37]:
df2 = df1.reset_index().melt(id_vars = 'ID',value_vars = ['S_1','S_2'],value_name = 'Math')
df2.loc[13:16]

,ID,School,Math
13,1304,S_1,85.2
14,1305,S_1,61.7
15,2101,S_1,NaN
16,2102,S_1,NaN


In [38]:
#去除缺失值之后比较
df2.set_index('ID').dropna().equals(df[['School','Math','ID']].set_index('ID'))

True

# 三、压缩与展开stack与unstack

**感觉stack和unstack都是在原有表的基础上进行操作，会保留表中的所有信息**

**而pivot_table和melt是可以选择表中的部分列进行操作的**

In [40]:
df_s = pd.pivot_table(df,index=['Class','ID'],columns='Gender',values=['Height','Weight'])
df_s.groupby('Class').head(2)

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
C_2   1201    NaN  188.0    NaN  68.0
      1202  176.0    NaN   94.0   NaN
C_3   1301    NaN  161.0    NaN  68.0
      1302  175.0    NaN   57.0   NaN
C_4   2401  192.0    NaN   62.0   NaN
      2402    NaN  166.0    NaN  82.0

## stack

In [41]:
#默认列索引由下而上展开
df_s.stack().groupby('Class').head(2)

Height  Weight
Class ID   Gender                
C_1   1101 M        173.0    63.0
      1102 F        192.0    73.0
C_2   1201 M        188.0    68.0
      1202 F        176.0    94.0
C_3   1301 M        161.0    68.0
      1302 F        175.0    57.0
C_4   2401 F        192.0    62.0
      2402 M        166.0    82.0

### 参数level可指定变化的列索引是哪一层（或哪几层，需要列表）

In [45]:
df_stacked = df_s.stack(level=0).groupby('Class').head(2)
df_stacked.head()

Gender              F      M
Class ID                    
C_1   1101 Height NaN  173.0
           Weight NaN   63.0
C_2   1201 Height NaN  188.0
           Weight NaN   68.0
C_3   1301 Height NaN  161.0

## unstack

展开，功能类似于pivot_table

In [51]:
df_stacked.unstack()

Gender          F             M       
           Height Weight Height Weight
Class ID                              
C_1   1101    NaN    NaN  173.0   63.0
C_2   1201    NaN    NaN  188.0   68.0
C_3   1301    NaN    NaN  161.0   68.0
C_4   2401  192.0   62.0    NaN    NaN

# 四、哑变量与因子化

## 1、Dummy Variable

### get_dummies函数，进行one_hot编码

In [54]:
#可选prefix参数添加前缀，prefix_sep添加分隔符
pd.get_dummies(df[['Class','Gender']],columns=['Class']).head()

,Gender,Class_C_1,Class_C_2,Class_C_3,Class_C_4
0,M,1,0,0,0
1,F,1,0,0,0
2,M,1,0,0,0
3,F,1,0,0,0
4,F,1,0,0,0


## 2、factorize

### 用于自然数编码，缺失值会被记做-1，sort参数表示是否排序后赋值

In [56]:
pd.factorize(['b', 'b', 'a', 'c',None, 'b'],sort = True)

(array([ 1,  1,  0,  2, -1,  1]), array(['a', 'b', 'c'], dtype=object))